In [1]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB

### Preprocessing

In [2]:
def euclidean_dist(a, b):
    return np.sqrt((b[0] - a[0]) ** 2 + (b[1] - a[1]) ** 2)

### Optimization Problems

In [2]:
def solve_master():

    # model
    master = gp.Model()

    # variables + objective function
    z = master.addVars(i, q, obj = f, vtype = GRB.BINARY, name = "z")
    eta = master.addVar(1, obj = 1)

    # model sense
    master.ModelSense = GRB.MINIMIZE

    # constraints (do I need Benders constraints at this point)
    master.addConstr(gp.quicksum(gp.quicksum(b[i, q] * z[i, q] for i in H) for q in Q) >= gp.quicksum(W[k] for k in K))
    master.addConstrs(gp.quicksum(z[i, q]) <= 1 for i in H)

    # update and solve the model
    master.update()
    master.optimize()

    # optimal solution
    for var in master.getVars():
        print(f"{var.VarName} = {var.x}")

    return None # master_opt_soln, master_opt_val

In [3]:
def solve_dsp():

    # model
    dsp = gp.Model()

    # variables + objective function
    alpha = dsp.addVars(k, obj = 1, name = "alpha")
    u = dsp.addVars(i, k, obj = z_hat, lb = 0, name = "u")
    v = dsp.addVars(i, obj = b * z_hat, lb = 0, name = "v")

    # model sense
    dsp.ModelSense = GRB.MAXIMIZE

    # constraints
    dsp.addConstrs(alpha[k] - u[a1][k] - u[a2][k] - W[k] * v[a1] <= F[a][k] for k in K for a in A_k_2)                             
    dsp.addConstrs(alpha[k] - u[a1][k] - W[k] * v[a1] <= F[a][k] for k in K for a in A_k_1)

    # update and solve the model
    dsp.update()
    dsp.optimize()

    # optimal solution
    for var in dsp.getVars():
        print(f"{var.VarName} = {var.x}")


    return None # dsp_opt_soln, dsp_opt_val

In [6]:
def solve_gap():

    # model
    gap = gp.Model()

    # variables + objective function
    x_hat = gap.addVars(a_i[k], k, obj = F, vtype = GRB.BINARY, name = "x_hat")

    # model sense
    gap.ModelSense = GRB.MINIMIZE

    # constraints
    gap.addConstrs(x[a_i[k]][k] == 1 for k in K)
    gap.addConstrs(gp.quicksum(W[k] * x[a_i[k]][k] for k in K) <= b[i][q_hat] for i in H(S_prime))

    # update and solve the model
    gap.update()
    gap.optimize()

    # optimal solution
    for var in gap.getVars():
        print(f"{var.VarName} = {var.x}")


    return None # gap_opt_soln, gap_opt_val


### Algorithms

In [7]:
def benders_decomp():
    upper_bound = 0
    t = 0
    dsp_extreme_pts = set()
    terminate = False

    while terminate == False:
        master_opt_soln, master_opt_val = solver_master() 

        if master_opt_val == upper_bound:
            terminate = True
        else:
            dsp_opt_soln, dsp_opt_val = solve_dual_subproblem()
            dsp_extreme_pts.add(dsp_opt_soln)

            if dsp_opt_val + ... < upper_bound:
                upper_bound = dsp_opt_val + ...

        t += 1

    return None # benders_opt_soln

In [12]:
# def exact_nmchlp():
#     master_opt_soln, master_opt_val = solver_master()
#     gap_opt_soln, gap_opt_val = solver_master()
#     upper_bound = min(min[...], gap_opt_soln, np.inf)

#     for i, q in zip(H(S*), Q_i(S*)):
#         z_iq = 0
#         master_opt_soln, master_opt_val = solver_master()

#         if master_opt_val > upper_bound:
#             z_iq = 1
        
#     for i, q in zip(H*, Q_i*):
#         z_iq = 1
#         master_opt_soln, master_opt_val = solver_master()

#         if master_opt_val > upper_bound:
#             Q_i* = Q_i*.pop(q)

#             if len(Q_i*) == 0:
#                 H* = H*.pop(i)

#     S = set((i, q) for i, q in zip(H*, Q_i*))

#     for S' subset S s.t. (19), (20) hold:
#         # Sovle GAP problem

#         if phi(GAP(z(S'))) < UB:
#             UB = phi(GAP(z(S')))
                           
#     return None